In [8]:
import numpy as np 
import pandas as pd 
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix , classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import config as creds
import psycopg2

In [9]:

#Set up connection to database
def connect():
    
    # Set up a connection to the postgres server.
    conn_string = "host="+ creds.PGHOST +" port="+ "5432" +" dbname="+ creds.PGDATABASE +" user=" + creds.PGUSER \
                  +" password="+ creds.PGPASSWORD
    
    conn = psycopg2.connect(conn_string)
    print("Connected!")

    # Create a cursor object
    cursor = conn.cursor()
    
    return conn, cursor

In [10]:
conn, cursor = connect()

Connected!


In [15]:
# Load the data

file_path = ('diabetes_012_health_indicators_BRFSS2015.csv')
df = pd.read_csv(file_path)
#print(df.shape)

#Convert diabetes to 1 and prediabetes to 0
def diabetes_binary(x, r):
    if x == r:
        return('1')
    if x < r:
        return('0')
    
df['Diabetes_012'] = df["Diabetes_012"].apply(diabetes_binary, args =[2])



df.head(20)

df.columns                      

Index(['Diabetes_012', 'HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker',
       'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies',
       'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth',
       'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age', 'Education',
       'Income'],
      dtype='object')

In [16]:
#Create the lifestyle df
socioeconomic_df = df[['Diabetes_012','NoDocbcCost','Sex','Age','Education','Income','AnyHealthcare']]

#Drops NA rows
socioeconomic_df.dropna()

,Diabetes_012,NoDocbcCost,Sex,Age,Education,Income,AnyHealthcare
0,0,0.0,0.0,9.0,4.0,3.0,1.0
1,0,1.0,0.0,7.0,6.0,1.0,0.0
2,0,1.0,0.0,9.0,4.0,8.0,1.0
3,0,0.0,0.0,11.0,3.0,6.0,1.0
4,0,0.0,0.0,11.0,5.0,4.0,1.0
...,...,...,...,...,...,...,...
253675,0,0.0,1.0,5.0,6.0,7.0,1.0
253676,1,0.0,0.0,11.0,2.0,4.0,1.0
253677,0,0.0,0.0,2.0,5.0,2.0,1.0
253678,0,0.0,1.0,7.0,5.0,1.0,1.0


In [17]:
#Split the data into features(x) and Target(y)
y = socioeconomic_df["Diabetes_012"]
X = socioeconomic_df.drop(['Diabetes_012'], axis = 1)

In [18]:
socioeconomic_df['Diabetes_012'] = socioeconomic_df['Diabetes_012'] .astype('int64')

C:\Users\kylen\miniconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [19]:
#Check data types
socioeconomic_df.dtypes

Diabetes_012       int64
NoDocbcCost      float64
Sex              float64
Age              float64
Education        float64
Income           float64
AnyHealthcare    float64
dtype: object

In [20]:
X.describe()

,NoDocbcCost,Sex,Age,Education,Income,AnyHealthcare
count,253680.000000,253680.000000,253680.000000,253680.000000,253680.000000,253680.000000
mean,0.084177,0.440342,8.032119,5.050434,6.053875,0.951053
std,0.277654,0.496429,3.054220,0.985774,2.071148,0.215759
min,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000
25%,0.000000,0.000000,6.000000,4.000000,5.000000,1.000000
50%,0.000000,0.000000,8.000000,5.000000,7.000000,1.000000
75%,0.000000,1.000000,10.000000,6.000000,8.000000,1.000000
max,1.000000,1.000000,13.000000,6.000000,8.000000,1.000000


In [21]:
y.value_counts()

0    218334
1     35346
Name: Diabetes_012, dtype: int64

In [22]:
#Split the data into test and training
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.50,random_state=1, stratify=y)
Counter(y_train)

Counter({'0': 109167, '1': 17673})

In [23]:
X_test

,NoDocbcCost,Sex,Age,Education,Income,AnyHealthcare
5404,0.0,0.0,12.0,6.0,8.0,1.0
134844,0.0,0.0,8.0,6.0,8.0,1.0
129255,0.0,0.0,11.0,6.0,6.0,1.0
224524,0.0,0.0,8.0,6.0,8.0,1.0
92752,0.0,0.0,9.0,6.0,7.0,1.0
...,...,...,...,...,...,...
103802,0.0,0.0,7.0,5.0,6.0,1.0
7476,0.0,0.0,4.0,6.0,8.0,1.0
96653,0.0,0.0,10.0,4.0,6.0,1.0
87326,0.0,1.0,8.0,4.0,7.0,1.0


In [24]:
#Initialize Logistic Regression Model
model = LogisticRegression(solver='lbfgs',
                                max_iter=200,
                                random_state=1)



In [25]:
#Fit the model
model.fit(X_train,y_train)

LogisticRegression(max_iter=200, random_state=1)

In [26]:
predictions = model.predict(X_test)
results = pd.DataFrame({"Prediction": predictions, "Actual": y_test})
results.head(10)



,Prediction,Actual
5404,0,0
134844,0,0
129255,0,0
224524,0,0
92752,0,0
151708,0,0
55470,0,1
30688,0,0
94938,0,0
245034,0,0


In [27]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, predictions)

0.8601466414380322

In [28]:
from imblearn.metrics import classification_report_imbalanced
matrix = confusion_matrix(y_test, predictions)
print(matrix)

[[109049    118]
 [ 17621     52]]


In [29]:
report = classification_report(y_test, predictions)
print(report)

              precision    recall  f1-score   support

           0       0.86      1.00      0.92    109167
           1       0.31      0.00      0.01     17673

    accuracy                           0.86    126840
   macro avg       0.58      0.50      0.47    126840
weighted avg       0.78      0.86      0.80    126840



   ### Oversampling

In [30]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)



In [31]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)



LogisticRegression(random_state=1)

In [32]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.6457264374174383

In [33]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[69077, 40090],
       [ 6032, 11641]], dtype=int64)

In [34]:
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.92      0.63      0.66      0.75      0.65      0.42    109167
          1       0.23      0.66      0.63      0.34      0.65      0.42     17673

avg / total       0.82      0.64      0.66      0.69      0.65      0.42    126840



In [35]:
create_table = """
    CREATE TABLE IF NOT EXISTS socioeconomic_df(
Diabetes_012      INTEGER
NoDocbcCost       FLOAT,
Sex               FLOAT,
Age               FLOAT,
Education         FLOAT,
Income            FLOAT,
AnyHealthcare     FLOAT
    )
    """

In [36]:
# Execute SQL Command and commit to DB
conn.commit()